# ML exercise to classify cats and dogs images

In [1]:
import collections

from ml.sklearn import ImageFeatureExtractor, SKLinearImageModel
from file_parsers import DirectoryParser

ModuleNotFoundError: No module named 'dask_ml'

In [3]:
input_directory = 'data/demo'
dir_parser = DirectoryParser(input_directory)
print(dir_parser)

Found 24 jpg/png file(s) out of 25 file(s).
Unsupported files: {'1.txt'}


In [ ]:
feature_extractor = ImageFeatureExtractor()
model = SKLinearImageModel(load_model=True)

for filepath in dir_parser.full_path_image_files:
        X, _y = feature_extractor.transform_image_to_dataset([filepath])
        # prediction = model.predict(X)
        # print(f"{filepath}: {prediction}")